In [1]:
using LinearAlgebra
#using DiffEqOperators
using LazyArrays
using BlockBandedMatrices, BandedMatrices, FillArrays, ArrayLayouts, SparseArrays
using LazyBandedMatrices

function _gaussseidel2(L, U, b, x=copy(b), y=copy(b), M=5)
    for _=1:M
        mul!(@view(y[1:end-1]) , U , @view(x[2:end]))
        y[end] = 0
        y .= b .- y
        x .= L\ y
    end
    x
end

function ham_2d_old(n, m, Vij)
    h = 1/n
    D = 0.0*BandedBlockBandedMatrix(Zeros(n^2,n^2), (Fill(n,n), Fill(n,n)), (1,1), (1,1))
    
    #Do block (1,1) diagonal entry
    view(D,Block(1,1))[band(0)] = Vij[1,:] .+ 2m
    view(D,Block(1,1))[band(1)] .= -1m
    view(D,Block(1,1))[band(-1)] .= -1m
    
    #Loop over diagonals and off-diagonals
    for K = 2:n
      view(D,Block(K,K))[band(1)] .= -1m
      view(D,Block(K,K))[band(0)] = Vij[K,:] .+ 2m
      view(D,Block(K,K))[band(-1)] .= -1m
      view(D,Block(K,K-1))[band(0)] .= -1m
      view(D,Block(K-1,K))[band(0)] .= -1m
    end
    D
end

function finitedifference_2d(n)
    h = 1/n
    D² = BandedMatrix(0 => Fill(-2,n), 1 => Fill(1,n-1), -1 => Fill(1,n-1))/h^2
    D_xx = BandedBlockBandedMatrix(Kron(D², Eye(n)))
    D_yy = BandedBlockBandedMatrix(Kron(Eye(n), D²))
    D_xx + D_yy
end

function finitedifference_2d_upper(n, D, U = copy(D))
    view(U,Block(1,1))[band(0)] = Fill(0, n)
    view(U,Block(1,1))[band(-1)] = Fill(0, n-1)
    
    for i = 2:n
        view(U,Block(i,i))[band(0)] = Fill(0, n)
        view(U,Block(i,i))[band(-1)] = Fill(0, n-1)
        view(U,Block(i,i-1))[band(0)] = Fill(0, n)
        view(U,Block(i,i-1))[band(-1)] = Fill(0, n-1)
        view(U,Block(i,i-1))[band(1)] = Fill(0, n-1)
    end
    U
end

#Feed in potential vector, D::BandedBlockBandedMatrix ,Ham::BandedBlockBandedMatrix, 
# m is the dt
#Returns what is almost the U1, U2 matrices 
function ham_2d(n, dt, Vij, D, Ham=copy(D))
    #add potential to the diagonal
    Ham = dt*1im/2.0*(Ham + Vij)
end

#Define U1, Vij is expected matrix format. F is the finite difference
#term from the spatial derivative, V is the on-site potential.
function U1(n, Vij, dt)
    F = finitedifference_2d(n,-dt/2);
    V = BandedMatrix(0 => dt/2.0.*Vij);
    U1 = BandedMatrix(I + F + V)
end

#Potential energy function for Harmonic Oscillator HO
function V_HO(x, y, w)
    V = w^2 .*(x.^2 .+ y.^2)
end

function get_Vij_HO(n, x_max, w, V_vec, V)
    dx = 2*x_max/(n-1);
    #V_mat = [V_HO(i,j,w) for i in -x_max:dx:x_max, j in -x_max:dx:x_max]
    
    for i in 1:n
        V_vec = V_HO(i,-x_max:dx:x_max,w)
        view(V,Block(i,i))[band(0)] = V_vec
    end
    V
end

function bi(i,j)
    return b = [exp(-(x-i)^2/σx^2 - (y-j)^2/σy^2) for x in 1:n, y in 1:n]
end

bi (generic function with 1 method)

In [41]:
n = 10;
x_max = 1;
w = 1.;
dt = 0.1;
V_vec = Zeros(n)
Vij = BandedBlockBandedMatrix(Kron(Eye(n),Eye(n)))
L̃ = 1.0im * spzeros(n^2,n^2)
v1 = rand(n^2)
v2 = copy(v1)
v2 = Int.(1:n^2)

#@time L̃ = sparse(BandedMatrix(0 => Complex.(Fill(1,n^2)), 
#        -1 => Complex.(Fill(1, n^2-1)), -n => Complex.(Fill(1, n*(n-1)))))
#promote_type(eltype(v2))
my_spm_diag1 = SparseMatrixCSC(n^2,n^2,v2[1:Int(n^2/2)],v2[1:Int(n^2/2)],v1)
#my_spm_diag2 = SparseMatrixCSC(4,4,v1,v2,v2)
#println(size(L̃))
#=
σx = 200;
σy = 200;
array = 1;
dn = n/array;
b = zeros(n,n)


#initial state
for i in 1:array, j in 1:array
    b += bi(dn*(i-1/2),dn*(j-1/2));
end

ψ_init = Complex.(reshape(b,n^2))

println("Preparing matrices...")

Deriv = finitedifference_2d(n)
Upper_Diagonal = copy(Deriv)
Upper_Diagonal = finitedifference_2d_upper(n,Deriv,Upper_Diagonal)

Ham = copy(Deriv)
Vij = get_Vij_HO(n,x_max,w,V_vec,Vij)
Ham = ham_2d(n,dt,Vij,Deriv,Ham)
U_1 = I + Ham
U_2 = I - Ham

@time L = sparse(U_1 - Upper_Diagonal)
#@time L = LowerTriangular(U_1)
#@time L1 = sparse(U_1)
#@time L̃ .= sparse(L)
#print(size(L̃))

U = sparse(BandedBlockBandedMatrix(UpperTriangular(@view(U_1[1:end-1,2:end])), ([fill(n,n-1); n-1], [n-1; fill(n,n-1)]),
                                        (0,1), (0,1)))

U2ψ = U_2*ψ_init
x = copy(U2ψ)
y = copy(U2ψ)

print("Beginning integration...")
@time ψ_fin = _gaussseidel2(L,U, U2ψ, x, y, 2)
=#

BoundsError: BoundsError: attempt to access 50-element Array{Int64,1} at index [101]

In [14]:
using PyPlot

pcolormesh(reshape(ψ_init,n,n))

Figure(PyObject <Figure size 640x480 with 1 Axes>)

PyObject <matplotlib.collections.QuadMesh object at 0x000000006883FBA8>